# scrapping
listé les adresses + numerode téléphone de tout les offices de tourisme sur le site : http://www.guidedutourisme.net/offices-de-tourisme.html<br>
but du doc : juste avoir une base hors ligne pour une personne ne possédant pas internet.<br>
Il va falloir lister les regions, puis ouvrir chaques page de region puis chaque page des offices de toursime<br>
Au final 2603 pages à ouvrir<br>.


In [1]:
# import des bibliotheques
import time
import requests
import pandas as pd
from bs4 import BeautifulSoup as bs

## récupération des adresse des page regions

In [2]:
# demander la page principale et vérifié le code (200 == OK) 
page_princ_request = requests.get(url='http://www.guidedutourisme.net/offices-de-tourisme.html')
print(page_princ_request.status_code)

200


In [3]:
#utilisé beautifulsoup pour parser la page
page_princ_soup = bs(page_princ_request.text,'html.parser')

In [4]:
#voir pour recupérer les adresses
lien_region = []
nom_region = []

#ne prendre que la premiere table
region = page_princ_soup.findAll('tr')[0]

#prendre que les balise contenant nom region et le lien
region = region.findAll('a') 

#ne prendre que le nom


for link in region:
    lien_region.append('http://www.guidedutourisme.net'+link.get('href'))
    nom_region.append(link.text)


## récupération des pages des offices de tourismes

In [5]:
#requete des pages (time sleep pour pas trop surcharger le serveurs)
page_region = {}
for lien,nom in zip(lien_region,nom_region):
    page_region[nom] = requests.get(lien)    
    time.sleep(1)

In [6]:
#transformer les pages avec beautifulsoup
page_region_soup = {}

for nom in nom_region:
    page_region_soup[nom] = bs(page_region[nom].text,'html.parser')

#page_region devient inutile
del page_region

In [7]:
lien_office = []
for nom in nom_region:
    for link in page_region_soup[nom].findAll('a',string="Consulter la fiche détaillée de l'organisme"):
        lien_office.append('http://www.guidedutourisme.net'+link.get('href'))
    

In [8]:
##requetes page des offices de tourisme
page_office = []
for link in lien_office:
    page_office.append(requests.get(link))
    time.sleep(0.2)

In [9]:
# transformer les pages avec beautifulsoup
office_soup = []
for page in page_office:
    office_soup.append(bs(page.text,'html.parser'))

In [10]:

office_soup[1].findAll('p')

[<p>Maison De La Vallee 114 Grand Rue<br/>67130 Schirmeck<br/><br/>150 Route De La Serva  <br/>67130 Belmont</p>,
 <p>03.88.47.18.51  - Schirmeck<br/><br/>03.88.97.39.50  - Belmont</p>,
 <p style="letter-spacing: 0.2px;line-height:1.7;margin-bottom:10px;color:#8c8c8c;font-family:Open sans;">Restez informer des dernières nouveautés de notre guide, des promos ainsi que des évenements.</p>]

In [11]:
office_soup[1].findAll('h3')[0].text

'67130 Schirmeck '

In [12]:
tel = []
adr = []
ville = []
for page in office_soup:
    ville.append(page.findAll('h3')[0].text)
    adr.append(page.findAll('p')[0].text)
    tel.append(page.findAll('p')[1].text)

# création du fichier

In [13]:
#faire le dataframe
csv_final = pd.DataFrame(list(zip(ville,tel,adr)),columns=['ville','telephone','adresse'])
#classé par ville
csv_final.sort_values(by='ville',inplace=True)

In [14]:
#exporter le fichier
csv_final.to_csv('fichier_office.csv',index=False)